In [20]:
# pip install sentencepiece
# pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [22]:
df = pd.read_csv('naver_shopping.txt', sep = "\t", header = None)
df.columns = ['label', 'text']

In [23]:
y = [0 if rate < 3 else 1 for rate in df.label] # 3점 미만이면 0(부정), 3점 초과이면 1(긍정)으로 라벨 생성

In [24]:
#평가 데이터셋 분리
X_train_val, X_test, y_train_val, y_test = train_test_split(df['text'].tolist(), y, random_state=0)

#학습, 검증 데이터셋분리
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=0)

print(len(X_train), len(X_val), len(X_test))
print(len(y_train), len(y_val), len(y_test))

112500 37500 50000
112500 37500 50000


### bert-base-multilingual-cased를 이용한 미세조정학습


In [47]:
#정확도 측정을 위한 준비 단계 (... 잘 모르겠음..)

# pip install datasets
import torch
from datasets import load_metric
metric = load_metric("accuracy")


# 정확도 계산 함수 생성

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compite(predictions=predictions, references=labels)

In [49]:
#
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

In [50]:
# 토큰화 준비&확인
from transformers import pipeline
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
print(tokenizer.tokenize('안녕하세요. 반갑습니다.'))

inputs = tokenizer('안녕하세요. 반갑습니다.')
print(inputs)

['안', '##녕', '##하', '##세', '##요', '.', '반', '##갑', '##습', '##니다', '.']
{'input_ids': [101, 9521, 118741, 35506, 24982, 48549, 119, 9321, 118610, 119081, 48345, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [51]:
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments

In [52]:
# 토큰화
train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors='pt')
val_input = tokenizer(X_val, truncation=True, padding=True, return_tensors='pt')
test_input = tokenizer(X_test, truncation=True, padding=True, return_tensors='pt')

In [53]:
# dataset 생성
train_dataset = OurDataset(train_input, y_train)
val_dataset = OurDataset(val_input, y_val)
test_dataset = OurDataset(test_input, y_test)

In [54]:
# bert-base-multilingual-cased 사전학습 모형으로부터 분류기 모형을 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [55]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
)

In [45]:
# Trainer 객체생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    compute_metrics=compute_metrics,
)

In [46]:
trainer.train()

MlflowException: Could not find experiment with ID 0

In [ ]:
trainer.save_model('questionDetection_model_v0.0.1')

In [ ]:
bbb=